In [54]:
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [55]:
df_for = pd.read_csv('datos/api_foursquare.csv')
df_ae = pd.read_csv('datos/df_aemet_final.csv')
df_muni = pd.read_csv('datos/municipios.csv')

In [56]:
df_ae.head(2)                   #actvidades y movidas

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la-id28001


In [57]:
try:

    conexion1 = psycopg2.connect(database='Forq',
                                    user= 'postgres',
                                    password='admin',
                                    host= 'localhost',
                                    port='5432')
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('La contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('Error de conexion')
    else:
        print(f'ocurrio el error {e}')

cursor = conexion1.cursor()

In [58]:
query_creacion_aemet ="""CREATE TABLE IF NOT EXISTS clima (
    fecha VARCHAR(100),
    cielo VARCHAR(100),
    temp_celsius INT,
    sensacion_termica INT,
    racha_max_kmh FLOAT,
    precipitacion_mm FLOAT,
    nieve_mm INT,
    humedad_relativa_pct FLOAT,
    prob_precipitacion_pct VARCHAR(100),
    prob_nieve_pct INT,
    prob_tormenta_pct INT,
    avisos VARCHAR(100),
    direccion_viento VARCHAR(10),
    velocidad_viento_kmh INT,
    localizacion_id VARCHAR(50) PRIMARY KEY
);"""
cursor.execute(query_creacion_aemet)
conexion1.commit()

In [59]:
df_ae.head(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la-id28001


In [60]:
lista_tuplas= [tuple(fila) for fila in df_ae.values]
tuple(df_ae.values[0])


('2024-08-29 12:00:00',
 'Nubes altas',
 21,
 21,
 25.0,
 0.0,
 0,
 69,
 '95',
 0,
 75,
 'Riesgo',
 'S',
 7,
 'acebeda-la-id28001')

In [61]:
for tupla in lista_tuplas:
    tupla = tuple(val[:50] if isinstance(val, str) else val for val in tupla)
    
    query_insercion_clima = """
    INSERT INTO clima (
        fecha, cielo, temp_celsius, sensacion_termica, racha_max_kmh, precipitacion_mm, 
        nieve_mm, humedad_relativa_pct, prob_precipitacion_pct, prob_nieve_pct, prob_tormenta_pct, 
        avisos, direccion_viento, velocidad_viento_kmh, localizacion_id
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (localizacion_id) DO NOTHING
    """
    valores = tupla
    cursor.execute(query_insercion_clima, valores)
    conexion1.commit()
    